In [1]:
import torch 
import numpy as np
import ipytest
ipytest.autoconfig()

In [2]:
def generate_bias(out_channels, bias):
    if bias:
        return torch.rand(out_channels)
    else:
        return torch.zeros(out_channels)

def generate_filter(in_channels, out_channels, kernel_size):
    if type(kernel_size) == tuple:
        return torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
    elif type(kernel_size) == int:
        return torch.rand(in_channels, out_channels, kernel_size, kernel_size)

def apply_dilation(total_mult, dilation):
    zero_tensor = torch.zeros(
        (total_mult.shape[0] - 1) * dilation + 1,
        (total_mult.shape[1] - 1) * dilation + 1
    )

    for a in range(0, zero_tensor.shape[0], dilation):
        for b in range(0, zero_tensor.shape[1], dilation):
            zero_tensor[a][b] = total_mult[a // dilation][b // dilation]

    return zero_tensor

def update_feature_map(feature_map, res, i, j, stride, dilation, filter):
    updated_map = np.add(
        feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
        j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1],
        res
    )
    feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
    j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1] = updated_map
    return feature_map

def process_padding(convolution, output_padding, padding):
    for t in range(len(convolution)):
        if output_padding > 0:
            padded = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
            convolution[t] = padded(convolution[t])

        convolution[t] = convolution[t][0 + padding:convolution[t].shape[0] - padding,
                         0 + padding:convolution[t].shape[1] - padding]

    return convolution

def ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def convolution(matrix):
        if padding_mode != 'zeros':
            raise Exception('Only "zeros" padding mode is supported in ConvTranspose2d')

        bias_values = generate_bias(out_channels, bias)
        filter = generate_filter(in_channels, out_channels, kernel_size)

        convolution = []

        for l in range(out_channels):
            feature_map = torch.zeros(
                (matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1
            )

            for c in range(in_channels):
                for i in range(0, matrix.shape[1]):
                    for j in range(0, matrix.shape[2]):
                        val = matrix[c][i][j]
                        total_mult = val * filter[c][l]
                        zero_tensor = apply_dilation(total_mult, dilation)
                        feature_map = update_feature_map(feature_map, zero_tensor, i, j, stride, dilation, filter)

            convolution.append(np.add(feature_map, np.full((feature_map.shape), bias_values[l])))

        convolution = process_padding(convolution, output_padding, padding)

        return convolution, filter, torch.tensor(bias_values)

    return convolution

In [3]:
def test_1():
    tensor2 = torch.rand(3, 18, 18)
    myFunction2 = ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_2():
    tensor2 = torch.rand(1, 11, 11)
    myFunction2 = ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_3():
    tensor2 = torch.rand(3, 10, 10)
    myFunction2 = ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

def test_4():
    tensor2 = torch.rand(2, 12, 12)
    myFunction2 = ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    result, kernel, bias_val = myFunction2(tensor2)
    torchFunction2 = torch.nn.ConvTranspose2d(in_channels=2, out_channels=2, kernel_size=3, stride=10, padding=0,output_padding=0, dilation=3, bias=True, padding_mode='zeros')
    torchFunction2.weight.data = kernel
    torchFunction2.bias.data = bias_val
    result_array = np.array(result) if isinstance(result, list) else result.numpy()
    assert np.allclose(result_array, torchFunction2(tensor2).data.numpy())

## Дополнительное задание

In [4]:
def apply_transpose_padding(matr, transp_stride, pad):
    zero_tensor = np.zeros((((matr.shape[0] - 1) * transp_stride) + 1, ((matr.shape[1] - 1) * transp_stride) + 1))
    
    for a in range(0, zero_tensor.shape[0], transp_stride):
        for b in range(0, zero_tensor.shape[1], transp_stride):
            zero_tensor[a][b] = matr[a // transp_stride][b // transp_stride]

    pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
    
    return pad_matr

def process_output_channel(matrix, filter, bias_values, in_channels, dilation, stride, l):
    feature_map = np.array([])

    for i in range(0, matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1) + 1, stride):
        for j in range(0, matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1) + 1, stride):
            total = 0

            for c in range(in_channels):
                val = matrix[c][i:i + (filter.shape[2] - 1) * dilation + 1:dilation,
                      j:j + (filter.shape[3] - 1) * dilation + 1:dilation]

                local_sum = (val * filter[l][c]).sum()
                total = total + local_sum

            feature_map = np.append(feature_map, float(total + bias_values[l]))

    return feature_map.reshape(
        (matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1)) // stride + 1,
        (matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1)) // stride + 1)

def select_padding_mode(padding_mode, padding):
    if padding_mode == 'zeros':
        return torch.nn.ZeroPad2d(padding)
    elif padding_mode == 'reflect':
        return torch.nn.ReflectionPad2d(padding)
    elif padding_mode == 'replicate':
        return torch.nn.ReplicationPad2d(padding)
    elif padding_mode == 'circular':
        pass
    else:
        raise ValueError(f"Unsupported padding mode: {padding_mode}")

def select_bias(bias, out_channels):
    if bias:
        return torch.rand(out_channels)
    else:
        return torch.zeros(out_channels)

def myTranspConv2dInverse(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def convolution(matrix):
        pad = kernel_size - 1
        result_matrix = []

        for matr in matrix:
            pad_matr = apply_transpose_padding(matr, transp_stride, pad)
            result_matrix.append(pad_matr)
        
        matrix = torch.tensor(result_matrix)
        bias_values = select_bias(bias, out_channels)
        pad = select_padding_mode(padding_mode, padding)
        matrix = pad(matrix)
        filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))
        filter_for_transpose = []
        
        for j in range(out_channels):
            filter_in = []
            for i in range(in_channels):
                filter_in.append(np.flip(np.array(filter[j][i])))
            filter_for_transpose.append(filter_in)
            
        filter_for_transpose = torch.tensor(filter_for_transpose)
        filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)
        stride = 1
        convolution_result = []

        for l in range(out_channels):
            feature_map = process_output_channel(matrix, filter, bias_values, in_channels, dilation, stride, l)
            convolution_result.append(feature_map)

        return np.array(convolution_result), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_values))

    return convolution

In [5]:
def run_test2(test_name, in_channels, out_channels, kernel_size, transp_stride, input_shape):
    tensor = torch.rand(*input_shape)
    my_function = myTranspConv2dInverse(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, transp_stride=transp_stride, bias=True)
    result, kernel, bias_val = my_function(tensor)
    
    torch_function = torch.nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=transp_stride, bias=True)
    torch_function.weight.data = kernel
    torch_function.bias.data = bias_val
    
    my_result = str(np.round(result, 2))
    torch_result = str(np.round(np.array(torch_function(tensor).data), 2))
    assert torch_result == my_result, f"Test {test_name} failed"


In [6]:
def test_5():
    run_test2("test_1", in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, input_shape=(3, 5, 6))

def test_6():
    run_test2("test_2", in_channels=1, out_channels=2, kernel_size=4, transp_stride=3, input_shape=(1, 28, 28))

def test_7():
    run_test2("test_3", in_channels=7, out_channels=1, kernel_size=3, transp_stride=1, input_shape=(7, 10, 10))

def test_8():
    run_test2("test_3", in_channels=7, out_channels=1, kernel_size=3, transp_stride=5, input_shape=(7, 10, 10))

In [7]:
ipytest.run()

........                                                                                     [100%]
======================================== warnings summary =========================================
t_4e210d878b79433fa961b2bd74f9418b.py::test_1
t_4e210d878b79433fa961b2bd74f9418b.py::test_2
t_4e210d878b79433fa961b2bd74f9418b.py::test_3
t_4e210d878b79433fa961b2bd74f9418b.py::test_4
  C:\Users\an501\AppData\Local\Temp\ipykernel_1288\3863638600.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
    return convolution, filter, torch.tensor(bias_values)

t_4e210d878b79433fa961b2bd74f9418b.py::test_5
  C:\Users\an501\AppData\Local\Temp\ipykernel_1288\3693573716.py:59: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before convertin

<ExitCode.OK: 0>